In [111]:
# import the necessary packages
import argparse
import os
from pprint import pprint

import cv2

import imutils
from imutils import paths

import keras
from keras.models import load_model
from keras import backend as K
from keras.preprocessing.image import array_to_img, img_to_array, load_img

import matplotlib as mpl
import matplotlib.pyplot as plt
import concurrent.futures
import glob

import numpy as np

from test_generator import test_generator
from keras.preprocessing.image import img_to_array
from sklearn import metrics

from utils import get_unused_dir_num
from test_utils import CvPutJaText

import json
import tensorflow as tf
tf.compat.v1.disable_eager_execution()


from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Activation

In [112]:

class NotFoundError(Exception):
    pass

def get_unused_result_dir_num(pref):
    os.makedirs('./results', exist_ok=True)
    dir_list = os.listdir(path='./results')
    for i in range(1000):
        search_dir_name = pref + "_" + '%03d' % i
        if search_dir_name not in dir_list:
            return search_dir_name
    raise NotFoundError('Error')


def _read_image_path_and_label_from_test_file(test_file):
    with open(test_file) as f:
        for line in f:
            yield line.split()


def create_polygon_json_from_mask(gray, filename, class_id=None):
    _, contours, hierarchy = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    line = filename

    for cnt in contours:
        if len(cnt) <= 2:
            continue

        cnt = cnt.flatten()

        line += " "
        line += ",".join(list(map(str, cnt)))
        if class_id is not None:
            line += "," + str(class_id)
    return line

In [148]:

def predict_images(model, class_names, image_path_list=None, image_path=None, imshow=False, predict_dir=None,
                   model_image_size=(28, 28), image_class=None):

    if image_path_list is None:
        image_path_list = list(paths.list_images(image_path)) if os.path.isdir(
            image_path) else [image_path]

    annotations = []

    preds = []
    for path in image_path_list:
        image = cv2.imread(path)
        modelsize_image = cv2.resize(image, model_image_size)
        output_image = imutils.resize(image, width=400)

        # pre-process the image for classification
        orig = cv2.resize(image, model_image_size).astype("float")
        image = orig / 255.0
        orig = cv2.cvtColor(img_to_array(orig), cv2.COLOR_BGR2RGB)
        image = np.expand_dims(img_to_array(image), axis=0)

        # classify the input image
        predict = model.predict(image)
        
        

        preds.append(predict[0])

    return preds



In [231]:
fontPIL = "Dflgs9.ttc"
font_path = fontPIL


model_path = 'logs/vegetables_000_lenet_005/model.01-0.64.hdf5'
test_data_path = 'test_data/vegetables_000/test_list.txt'
model_image_size = (28, 28)

model_path = 'logs/smoke_000_inception_v3_000/model.68-0.18.hdf5'
test_data_path = 'test_data/smoke_000/test_list.txt'
model_image_size = (299, 299)

model_path = 'logs/smoke2_000_lenet_000/model.97-0.36.hdf5'
test_data_path = 'test_data/smoke2_000/test_list.txt'
model_image_size = (51, 28)

# model_path = 'logs/smoke2_000_inception_v3_001/model.20-0.64.hdf5'
# test_data_path = 'test_data/smoke2_000/test_list.txt'
# model_image_size = (299, 299)

model = load_model(model_path)

In [232]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_input (InputLayer)    [(None, 28, 51, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 51, 20)        1520      
_________________________________________________________________
activation (Activation)      (None, 28, 51, 20)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 25, 20)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 25, 50)        25050     
_________________________________________________________________
activation_1 (Activation)    (None, 14, 25, 50)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 12, 50)         0     

In [188]:
model_dir = os.path.dirname(model_path)
test_data_dir = os.path.dirname(test_data_path)
with open(os.path.join(model_dir, "config.json")) as f:
    config = json.load(f)
class_path = os.path.join(model_dir, "classes.txt")
with open(class_path) as fp:
    class_names = [line.strip() for line in fp]
CLASS = len(class_names)

In [189]:

inp = model.layers[0].input
out = model.layers[-2].output
predictions = Dense(2, activation='sigmoid', name='dense_2')(out)
model_sigmoid = Model(inp, predictions)
# model_sigmoid.summary()

In [191]:

# test_data_path = 'test_data/smoke2_000/train_list_2.txt'
test_path = test_data_path
contents = _read_image_path_and_label_from_test_file(test_path)
# result[l1][l2] : the number of images witch is predicted as l1, the true label is l2
result = np.zeros((CLASS, CLASS), dtype=int)

labels = []
predicts = []
pred_labs = []
stats = []

image_path_list = []
for content in contents:
    image_path_list.append(content[0])
    labels.append(int(content[1]))

predicts = predict_images(model=model, image_path_list=image_path_list,
                          class_names=class_names, predict_dir=predict_dir, model_image_size=model_image_size)
predicts = np.array(predicts)


predicts_sigmoid = predict_images(model=model_sigmoid, image_path_list=image_path_list,
                          class_names=class_names, predict_dir=predict_dir, model_image_size=model_image_size)
predicts_sigmoid = np.array(predicts_sigmoid)

In [195]:
predicts[1]

array([0.60957724, 0.3904228 ], dtype=float32)

In [196]:
len(predicts)

550

In [228]:
true_labs = []
img_labs = []

result = np.zeros((CLASS, CLASS), dtype=int)
cnt_labels = np.zeros((CLASS), dtype=int)
thre_score = 0.01
# thre_score = 0.001
# thre_score = None
# pn = np.zeros((CLASS, 2, 2), dtype=int)
# print(predicts[0])
# print(labels[0])
u, cnt_labels = np.unique(labels, return_counts=True)
print(u, cnt_labels)
for i, (pred, lab, image_path) in enumerate(zip(predicts, labels, image_path_list)):
# for i, (pred, lab, image_path) in enumerate(zip(predicts_sigmoid, labels, image_path_list)):
#     cnt_labels[lab] += 1
    if thre_score == None:
        argmax = pred.argmax()
        pred_labs.append(argmax)
        true_labs.append(lab)
        img_labs.append(image_path)
        result[argmax][lab] += 1
    else:
        for j, p in enumerate(pred):
            if p > thre_score:
                pred_labs.append(p)
                true_labs.append(lab)
                img_labs.append(image_path)
                result[j][lab] += 1
print(result)
print(cnt_labels)
all_sum = len(predicts)
# col_sum[l] : count of whose true label is l
col_sum = np.sum(result, axis=0)
# row_sum[l] : count of whose predicted label is l
row_sum = np.sum(result, axis=1)
all_score = []
all_true = []
for i in range(CLASS):
    recall = float(result[i][i]) / cnt_labels[i] if col_sum[i] != 0 else -1
    precision = float(result[i][i]) / row_sum[i] if row_sum[i] != 0 else -1
    specificity = float(
        all_sum + result[i][i] - col_sum[i] - row_sum[i]) / (all_sum - col_sum[i])
    f_value = 2 * recall * precision / \
        (recall + precision) if recall != -1 and precision != -1 else -1
    accuracy = (all_sum + 2 * result[i][i] - row_sum[i] -
                col_sum[i]) / all_sum if all_sum != 0 else -1
    print('accuracy',accuracy,'recall', recall, 'precision', precision)
    print('誤検知',1-precision,'検知漏れ', 1-recall)

[0 1] [289 261]
[[289 232]
 [205 216]]
[289 261]
accuracy 0.20545454545454545 recall 1.0 precision 0.5547024952015355
誤検知 0.4452975047984645 検知漏れ 0.0
accuracy 0.20545454545454545 recall 0.8275862068965517 precision 0.5130641330166271
誤検知 0.4869358669833729 検知漏れ 0.1724137931034483
